<h1><center><font size="6">Jigsaw Unintended Bias in Toxicity Classification</font></center></h1>
<h1><center><font size="6">Simple EDA</font></center></h1>

<center><img src="https://d1f5hsy4d47upe.cloudfront.net/36/36f0885cfc92b2d8abf1073f229c0047_w.jpg" width="300"></img></center>

<br>

# <a id='0'>Content</a>

- <a href='#1'>Introduction</a>  
- <a href='#2'>Data exploration</a>  
    - <a href='#20'>Data Overview</a>  
    - <a href='#21'>ID</a>  
    - <a href='#22'>target</a>  
    - <a href='#23'>comment_text</a>  
    - <a href='#24'>comment_text : value_count</a>  
    - <a href='#25'>comment_text : comment_length</a>  
    - <a href='#26'>comment_text : Number of words</a>  
    - <a href='#27'>comment_text : Number of Capitals</a>
    - <a href='#28'>comment_text : Number of exclamation marks</a>
    - <a href='#29'>comment_text : Wordcloud</a>

# <a id='#1'>Introduction</a>   

*Work still progress.*

Firstly,I used  below kernel as a reference. Thanks!

・https://www.kaggle.com/gpreda/jigsaw-eda

・https://www.kaggle.com/nz0722/simple-eda-text-preprocessing-jigsaw

## Backgorund

In the past competition, we made the model for perdicting "target" from Test data, and minimize this type of unintended bias.<br>

e.g.  "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"<br>
 →terget is 0.000000<br>
 
e.g.  "haha you guys are a bunch of losers."<br>
 →terget is 0.89617, because "loser" may be toxicity comment.  <br>
 
&emsp;

But,that'model has unintende bias, and we have to remove this.  <br>

e.g.  "I'm a gay woman."<br>
 
 
This sentence is not toxic, but old model may be deciding toxicity comment.<br>

&emsp;

So, let' make better model for removing unintend bias!<br>
 
## Feature of my kernel

In this kernel, I conclude whether all variables are useful for making new variable or not as far as possible. 



## Conclusion

In this paragraph, I summarizes what I found in this kernel.
It will be updated as appropriate.


・ "test.csv" has only comment text. So I think we can make new variables only about comment_text.(I'm not confident・・・)

・New variable　"Number of words"　"Number of Capitals"　"Number of exclamation marks" may be useful.

・In the comment data,there is something frong.For example "Trump" at highly insult, "Damn" at highly obscene.

・


Import
------

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.tokenize import TweetTokenizer
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from wordcloud import WordCloud
from collections import Counter
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
pd.set_option('max_colwidth',400)
pd.set_option('max_columns', 50)
import json
import altair as alt
from  altair.vega import v3
from IPython.display import HTML
import gc
import os

In [ ]:
# Preparing altair. I use code from this great kernel: https://www.kaggle.com/notslush/altair-visualization-2018-stackoverflow-survey

vega_url = 'https://cdn.jsdelivr.net/npm/vega@' + v3.SCHEMA_VERSION
vega_lib_url = 'https://cdn.jsdelivr.net/npm/vega-lib'
vega_lite_url = 'https://cdn.jsdelivr.net/npm/vega-lite@' + alt.SCHEMA_VERSION
vega_embed_url = 'https://cdn.jsdelivr.net/npm/vega-embed@3'
noext = "?noext"

paths = {
    'vega': vega_url + noext,
    'vega-lib': vega_lib_url + noext,
    'vega-lite': vega_lite_url + noext,
    'vega-embed': vega_embed_url + noext
}

workaround = """
requirejs.config({{
    baseUrl: 'https://cdn.jsdelivr.net/npm/',
    paths: {}
}});
"""

#------------------------------------------------ Defs for future rendering
def add_autoincrement(render_func):
    # Keep track of unique <div/> IDs
    cache = {}
    def wrapped(chart, id="vega-chart", autoincrement=True):
        if autoincrement:
            if id in cache:
                counter = 1 + cache[id]
                cache[id] = counter
            else:
                cache[id] = 0
            actual_id = id if cache[id] == 0 else id + '-' + str(cache[id])
        else:
            if id not in cache:
                cache[id] = 0
            actual_id = id
        return render_func(chart, id=actual_id)
    # Cache will stay outside and 
    return wrapped
            
@add_autoincrement
def render(chart, id="vega-chart"):
    chart_str = """
    <div id="{id}"></div><script>
    require(["vega-embed"], function(vg_embed) {{
        const spec = {chart};     
        vg_embed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
        console.log("anything?");
    }});
    console.log("really...anything?");
    </script>
    """
    return HTML(
        chart_str.format(
            id=id,
            chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
        )
    )

HTML("".join((
    "<script>",
    workaround.format(json.dumps(paths)),
    "</script>",
)))

In [ ]:
print(os.listdir("../input"))

# <a id='2'>Data exploration</a>  

# <a id='20'>Data Overview</a>  

Let's have a quick look at the data first

In [ ]:
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

Firstly, train_data has **id** and **target**. 
* **id**: unique number for comment.
* **target**: express toxicity determined by rater

Secondly, the **comment_text** are stored in `train` and `test`.  
* **comment_text**

In `train`, following feature expresses aggression of comment.
* **severe_toxicity**
* **obsence**
* **identity_attack**
* **insult**

The topic is also related to five categories: race or ethnicity, gender, sexual orientation, religion, disability, as following:
* **race or ethnicity**: asian, black, jewish, latino, other_race_or_ethnicity, white  
* **gender**: female, male, transgender, other_gender  
* **sexual orientation**: bisexual, heterosexual, homosexual_gay_or_lesbian, other_sexual_orientation  
* **religion**: atheist,buddhist,  christian, hindu, muslim, other_religion  
* **disability**: intellectual_or_learning_disability, other_disability, physical_disability, psychiatric_or_mental_illness  

We also have few article/comment identification information:
* **created_date**  
* **publication_id**   
* **parent_id**  
* **article_id** 

Several user feedback information associated with the comments are provided:
* **rating**  
* **funny**  
* **wow**  
* **sad**  
* **likes**  
* **disagree**  
* **sexual_explicit**  

In the datasets are also 2 fields relative to annotations:
* **identity_annotator_count**  
* **toxicity_annotator_count**

In [ ]:
#We may need Time series analisys because train-data has created_date.
#What is "intellectual_or_learning_disability","rating","sexual_explicit","identity_annotator_count","toxicity_annotator_count","publication_id"? I have to research them.

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.describe()

In [ ]:
test.describe()

# <a id='21'>ID</a>  

## Data exploration
Let's have a look at the data first

In [ ]:
train["id"].value_counts().head()

・　ID has unique number.

In [ ]:
train_id = train["id"].astype(int)
plt.subplots(figsize=(10, 7)) 
sns.set_context("poster")
sns.distplot(train_id,kde =False)
plt.title('Train ID Histogram')

In [ ]:
train["id"].min()

In [ ]:
train["id"].max()

・　Train ID has nearly 500000 and 5500000

In [ ]:
test_id = test["id"].astype(int)
plt.subplots(figsize=(10, 7)) 
sns.set_context("poster")
sns.distplot(test_id,kde =False)
plt.title('Test ID Histogram');

In [ ]:
sns.set_context("poster")
train['created_date'] = pd.to_datetime(train['created_date']).values.astype('datetime64[M]')
plt.subplots(figsize=(15, 6))
ax1 = sns.lineplot(x=train['created_date'], y=train['id'],label='Train ID',color = "red");
ax2 = ax1.twinx()
sns.lineplot(x=train['created_date'], y=train['target'], label='target', ax=ax2);
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2, loc='lower right');

## Discussion
ID increase with time, and target also increase, but I think they has no corelation between ID and target.

## Conclusion

I think we can't make new valuable from ID.

# <a id='22'>target</a>  

## Data exploration
Let's have a look at the data first

In [ ]:
hist_df = pd.cut(train['target'], 20).value_counts().sort_index().reset_index().rename(columns={'index': 'bins'})
hist_df['bins'] = hist_df['bins'].astype(str)
render(alt.Chart(hist_df).mark_bar().encode(
    x=alt.X("bins:O", axis=alt.Axis(title='Target')),
    y=alt.Y('target:Q', axis=alt.Axis(title='Count')),
    tooltip=['target', 'bins']
).properties(title="Counts of target bins", width=1000, height=500).interactive())


In [ ]:
print("(target=0)/all_target = ",(train['target'] >= 0).sum() / train.shape[0])
print("(target>0)/all_target = ",(train['target'] > 0).sum() / train.shape[0])
print("(target>0.25)/all_target = ",(train['target'] >= 0.25).sum() / train.shape[0])
print("(target>=0.5)/all_target ",(train['target'] >= 0.5).sum() / train.shape[0])

In [ ]:
train['target'].value_counts().head(20)

## Discussion
We understood "target" distribution.

・There are many 0 target.

・Why are there so many 0.15-0.20 target? We must research it later.

## Conclusion

I think we can't make new valuable from target, because target is objective variable.

## <a id='23'>comment_text</a>  

**Let's check the 'comment_text' about each new variable!**

## <a id='24'>comment_text : value_count</a>  

## Data exploration

In [ ]:
train['comment_text'].value_counts().head(50)

In [ ]:
test['comment_text'].value_counts().head(50)

## Discussion
Hum・・・.Why so many same comment　"START WORKING ・・・" have train and test data? May this be commercial comment？ 

We have to research these later.

## Conclusion

later.

## <a id='25'>comment_text : comment_length</a>  

## Data exploration

In [ ]:
train_plus = train
train_plus['train_comment_length'] = train.comment_text.apply(len)
test_plus = test
test_plus['test_comment_length'] = test.comment_text.apply(len)

In [ ]:
sns.set_context("poster");
plt.figure(figsize=(15, 8));
sns.distplot(train_plus['train_comment_length'] ,kde=False, rug=False,bins=100,norm_hist=True,label = "train");
sns.distplot(test_plus['test_comment_length'] ,kde=False, rug=False,bins=100,norm_hist=True,label = "test",axlabel = "comment_length");
plt.legend();
plt.title("Histgram of train adn test comment_length with normalize");

This is histogram of train_comment length.

We can see rise around 1000.

In [ ]:
plt.figure(figsize=(8, 8));
sns.scatterplot(x=train_plus['train_comment_length'], y=train_plus['target'],alpha = 0.5);
plt.title('Target & Train_comment_length');

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['train_comment_length'],train_plus['target'])[1,0])

Next , we check outlier using boxplot.

In [ ]:
plt.figure(figsize=(3, 8));
sns.boxplot(y=train_plus['train_comment_length'] )
plt.title("Boxplot of train_comment_length");

In [ ]:
#Check the Number of 75% point
q25,q50,q75= np.percentile(train_plus['train_comment_length'], [25,50,75])
iqr = q75- q25
q100 = q75 + iqr*1.5
print("Nuber of 25% point = ",q25)
print("Nuber of 50% point = ",q50)
print("Nuber of 75% point = ",q75)
print("Nuber of 100% point = ",q100)

Let's check the train_data more than 100% point

In [ ]:
train_plus[train_plus['train_comment_length'] > 894].head()

In [ ]:
train_plus['comment_text'][train_plus['train_comment_length'] > 414].value_counts().head()

In [ ]:
plt.figure(figsize=(8, 8));
sns.scatterplot(x=train_plus['train_comment_length'][train_plus['train_comment_length'] > 894], y=train_plus['target'][train_plus['train_comment_length'] > 894],alpha = 0.5);
plt.title('Target & Train_comment_length');

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['train_comment_length'][train_plus['train_comment_length'] > 894],train_plus['target'][train_plus['train_comment_length'] > 894])[1,0])

## Discussion

Correlation of comment_length is 0.006, and Cor of comment_length > 894 is 0.011.

I think the longer the comment, the more likely it is that the comment is toxic.

## Conclusion
We can see  defined correlation between "target" and "comment_length".

So I think this variable may be useful.

## <a id='26'>comment_text : Number of words</a>  

In [ ]:
train_plus['num_words'] = train['comment_text'].apply(lambda comment: len(comment.split()))

In [ ]:
plt.figure(figsize=(8, 8));
sns.scatterplot(x=train_plus['num_words'], y=train_plus['target'],alpha = 0.5);
plt.title('Target & Number of words');

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['num_words'],train_plus['target'])[1,0])

In [ ]:
plt.figure(figsize=(3, 8));
sns.boxplot(y=train_plus['num_words'] )
plt.title("Boxplot of num_words");

In [ ]:
#Check the Number of 100% point
q25,q50,q75= np.percentile(train_plus['num_words'], [25,50,75])
iqr = q75- q25
q100 = q75 + iqr*1.5
print("Nuber of 25% point = ",q25)
print("Nuber of 50% point = ",q50)
print("Nuber of 75% point = ",q75)
print("Nuber of 100% point = ",q100)

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['num_words'][train_plus['num_words'] > 156],train_plus['target'][train_plus['num_words'] > 156])[1,0])

## Discussion

Correlation between traget and "num_words" is 0.0096, and Cor between target and "num_words > 156" is 0.017.

I think the longer the num_words, the more likely it is that the comment is toxic.

## Conclusion
We can see  defined correlation between "target" and "num_words".

So I think this variable may be useful.

## <a id='27'>comment_text : Number of Capitals</a>

In [ ]:
train_plus['capitals'] = train['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))

In [ ]:
plt.figure(figsize=(8, 8));
sns.scatterplot(x=train_plus['capitals'], y=train_plus['target'],alpha = 0.5);
plt.title('Target & Number of capitals');

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['capitals'],train_plus['target'])[1,0])

In [ ]:
plt.figure(figsize=(3, 8));
sns.boxplot(y=train_plus['capitals'] )
plt.title("Boxplot of capitals");

In [ ]:
#Check the Number of 100% point
q25,q50,q75= np.percentile(train_plus['capitals'], [25,50,75])
iqr = q75- q25
q100 = q75 + iqr*1.5
print("Nuber of 25% point = ",q25)
print("Nuber of 50% point = ",q50)
print("Nuber of 75% point = ",q75)
print("Nuber of 100% point = ",q100)

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['capitals'][train_plus['capitals'] > 23],train_plus['target'][train_plus['capitals'] > 23])[1,0])

## Discussion

Correlation between traget and "capitals" is 0.026, and Cor between target and "capitals > 23" is 0.064.

I think the longer the number of capitals, the more likely it is that the comment is toxic.

## Conclusion
We can see  defined correlation between "target" and "capitals".

So I think this variable may be useful stongly.

## <a id='28'>comment_text : Number of exclamation marks</a>

In [ ]:
train_plus['num_exclamation_marks'] = train['comment_text'].apply(lambda comment: comment.count('!'))

In [ ]:
plt.figure(figsize=(8, 8));
sns.scatterplot(x=train_plus['num_exclamation_marks'], y=train_plus['target'],alpha = 0.5);
plt.title('Target & Number of num_exclamation_marks');

In [ ]:
print("correlation coefficients is ",np.corrcoef(train_plus['num_exclamation_marks'],train_plus['target'])[1,0])

In [ ]:
plt.figure(figsize=(3, 8));
sns.boxplot(y=train_plus['num_exclamation_marks'] )
plt.title("Boxplot of num_exclamation_marks");

In [ ]:
#Check the Number of 100% point
q25,q50,q75= np.percentile(train_plus['num_exclamation_marks'], [25,50,75])
iqr = q75- q25
q100 = q75 + iqr*1.5
print("Nuber of 25% point = ",q25)
print("Nuber of 50% point = ",q50)
print("Nuber of 75% point = ",q75)
print("Nuber of 100% point = ",q100)

## Discussion

Correlation between traget and "num_exclamation_marks" is 0.055.

I think the longer the number of num_exclamation_marks, the more likely it is that the comment is toxic.

## Conclusion
We can see  defined correlation between "target" and "num_exclamation_marks".

So I think this variable may be useful.

We can see a little correlation between target and New Variable.

## <a id='29'>comment_text : Wordcloud</a>

In [ ]:
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=50,
        max_font_size=40, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(10,10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(train['comment_text'].sample(20000), title = 'Prevalent words in comments - train data')

In [ ]:
show_wordcloud(test['comment_text'].sample(20000), title = 'Prevalent words in comments - test data')

*Politically related words are found*. For example Trump,Republician.

In [ ]:
show_wordcloud(train.loc[train['insult'] < 0.25]['comment_text'].sample(20000), 
               title = 'Prevalent comments with insult score < 0.25')

In [ ]:
show_wordcloud(train.loc[train['insult'] > 0.75]['comment_text'].sample(20000), 
               title = 'Prevalent comments with insult score > 0.75')

Many words of direct insult are found. For example,stupid,idot.

But, We can also find politically related words .for example Trump,President.

In [ ]:
show_wordcloud(train.loc[train['threat'] < 0.25]['comment_text'], 
               title = 'Prevalent words in comments with threat score < 0.25')

In [ ]:
show_wordcloud(train.loc[train['threat'] > 0.75]['comment_text'], 
               title = 'Prevalent words in comments with threat score > 0.75')

Many words of direct threat are found. For kill,shoot,head.

But, dog is also threat word? I have to research it later.

In [ ]:
show_wordcloud(train.loc[train['obscene']< 0.25]['comment_text'], 
               title = 'Prevalent words in comments with obscene score < 0.25')

In [ ]:
show_wordcloud(train.loc[train['obscene'] > 0.75]['comment_text'], 
               title = 'Prevalent words in comments with obscene score > 0.75')

Many words of direct obscene are found. ass.

Also, I was surprised that the following words would become obscene
Damn,fucking,shit,etc・・・.

In [ ]:
show_wordcloud(train.loc[train['target'] < 0.25]['comment_text'], 
               title = 'Prevalent words in comments with target score < 0.25')

In [ ]:
show_wordcloud(train.loc[train['target'] > 0.75]['comment_text'], 
               title = 'Prevalent words in comments with target score > 0.75')

# Discussion

Firstly, there is many Politically related words.

Secondly,We found many word along each feature,
but there is something frong,for example "Trump" at highly insult, "Damn" at highly obscene.

# Conclusion

later.

*Work still progress*